In [ ]:
!pip install functorch
print("--> Restarting colab instance") 
get_ipython().kernel.do_shutdown(True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:46tcmalloc: large alloc 1147494400 bytes == 0x65c10000 @  0x7f94385f6615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████████████████| 890.2 MB 5.2 kB/s 
     |████████████████████████████████| 557.1 MB 12 kB/s 
     |████████████████████████████████| 317.1 MB 35 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 849 kB 61.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolve

{'status': 'ok', 'restart': True}

In [ ]:
import matplotlib.pyplot as plt
import torch
from scipy.integrate import solve_ivp
from torch import nn
import numpy as np
%matplotlib inline

In [101]:
from functorch import vmap, vjp
from functorch import jacrev, jacfwd

# class NNApproximator(nn.Module):
#   def __init__(self, dim_input = 1, dim_output = 2, num_hidden = 2, dim_hidden = 1, activation=nn.Tanh()):
#     super().__init__()

#     self.layer_in = nn.Linear(dim_input, dim_hidden)
#     self.layer_out = nn.Linear(dim_hidden, dim_output)
#     # self.A = nn.Parameter(torch.randn(2,2))
#     self.k = nn.Parameter(torch.rand(1, requires_grad=True))
#     # self.A = self.k * torch.from_numpy(np.array([[-1,1],[1,-1]]))

#     num_middle = num_hidden - 1
#     self.middle_layers = nn.ModuleList(
#         [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
#     )
#     self.activation = activation

#   def forward(self, x):
#     out = self.activation(self.layer_in(x))
#     for layer in self.middle_layers:
#       out = self.activation(layer(out))
#     return self.layer_out(out)

#   # reference for implementing derivatives for batched inputs
#   # https://pytorch.org/functorch/stable/notebooks/jacobians_hessians.html
#   def jacobian(self, x):
#     jac = vmap(jacrev(self.forward))
#     return jac(x).squeeze()

In [ ]:
class NNOracle(nn.Module):
  def __init__(self, dim_input = 6, dim_output = 2):
    super().__init__()

    self.k = 50.0
    self.g = torch.from_numpy(np.array([[0, -9.81]]))
    self.L0 = 5.0

  def forward(self, x):
    x1 = x[:2]
    x2 = x[2:4]
    x3 = x[4:]
    dx1 = x2 - x1
    dx2 = x3 - x2
    dx1_norm = torch.sqrt(torch.sum(dx1 ** 2))
    dx2_norm = torch.sqrt(torch.sum(dx2 ** 2))
    f1 = -self.k * (dx1_norm - self.L0) * (dx1 / dx1_norm)
    f2 = self.k * (dx2_norm - self.L0) * (dx2 / dx2_norm)
    return self.g + f1 + f2

  def jacobian(self, x):
    jac = vmap(jacrev(self.forward))
    return jac(x).squeeze()

In [ ]:
def compute_data_loss(model, x_tr, y_tr):
  return 0.5 * torch.mean((model.forward(x_tr) - y_tr) ** 2)

def compute_PINN_loss(model, x, k):
    F_dot = model.jacobian(x)
    s1 = x[:, 0:2] - x[:, 2:4]
    s2 = x[:, 4:6] - x[:, 2:4]

    s1rot = s1 @ torch.from_numpy(np.array([[0, -1], [1, 0]]).T).float()
    s2rot = s2 @ torch.from_numpy(np.array([[0, -1], [1, 0]]).T).float()

    f1_constr = torch.norm(torch.einsum('ijk,ik->ij', F_dot[:, :, 0:2], s1), dim=1) / torch.norm(s1, dim=1)
    f2_constr = torch.norm(torch.einsum('ijk,ik->ij', F_dot[:, :, 4:6], s2), dim=1) / torch.norm(s2, dim=1)

    f1_perp = torch.einsum('ijk,ik->ij', F_dot[:, :, 0:2], s1rot) / (torch.norm(s1, dim=1) ** 2)[:, None]
    # print(f1_perp.shape)
    # print(torch.norm(s1, dim=1).shape)
    f2_perp = torch.einsum('ijk,ik->ij', F_dot[:, :, 4:6], s2rot) / (torch.norm(s2, dim=1) ** 2)[:, None]

    f1_perp = torch.einsum("ij,ik->i", f1_perp, s1)
    f2_perp = torch.einsum('ij,ik->i', f2_perp, s2)

    print(f1_constr)
    print(f2_constr)
    print(f1_perp)
    print(f2_perp)
    
    return ((k - f1_constr) ** 2).mean() + ((k - f2_constr) ** 2).mean() + (f1_perp ** 2).mean() + (f2_perp ** 2).mean()

In [ ]:
model = NNOracle()

Nt = 2
x = torch.rand(Nt, 6, requires_grad=True)
# print(x)
print(model.jacobian(x))
# print(compute_PINN_loss(model, x, model.k))

tensor([[[-215.8301,   49.9283,  576.3270, -517.4025, -360.4968,  467.4741],
         [  49.9284,   40.6224, -517.4025,  441.7377,  467.4741, -482.3601]],

        [[-501.8471,  -83.9989,  741.0087,  -79.9194, -239.1616,  163.9182],
         [ -83.9989,   37.2142,  -79.9194,    5.7067,  163.9182,  -42.9209]]],
       grad_fn=<SqueezeBackward0>)


In [ ]:
# x = torch.from_numpy(np.array([[0,0, 2,0, 3,2]])).float().requires_grad_()
x = torch.rand(size=(1,6), requires_grad=True)

F_dot = model.jacobian(x).detach()
F_dot_2 = F_dot[:,4:]
eigval, eigvec = torch.linalg.eig(F_dot_2)
eigvec = eigvec / eigvec[0,1]
# print(eigval)
# print(eigvec)

s2 = x[:, 4:6] - x[:, 2:4]
s2 = s2 / torch.norm(s2,dim=1)
print("This should be equal to k: ", s2 @ F_dot_2 @ s2.T - model.k)
print("This should be equal to k: ", s2.squeeze().reshape(2,1).T @ F_dot_2 @ s2.squeeze().reshape(2,1) - model.k)

s2rot = s2 @ torch.from_numpy(np.array([[0, -1], [1, 0]]).T).float().detach()
print("This should be zero...?", s2 @ F_dot_2 @ s2rot.T)
print("This should be zero...?", s2.squeeze().reshape(2,1).T @ F_dot_2 @ s2rot.squeeze().reshape(2,1))

print(s2.squeeze())
print(s2rot.squeeze())
print(F_dot_2.squeeze())


This should be equal to k:  tensor([[-6.8665e-05]], grad_fn=<SubBackward0>)
This should be equal to k:  tensor([[-6.8665e-05]], grad_fn=<SubBackward0>)
This should be zero...? tensor([[9.5367e-07]], grad_fn=<MmBackward0>)
This should be zero...? tensor([[7.1384e-07]], grad_fn=<MmBackward0>)
tensor([0.9762, 0.2167], grad_fn=<SqueezeBackward0>)
tensor([-0.2167,  0.9762], grad_fn=<SqueezeBackward0>)
tensor([[  25.9062,  108.5577],
        [ 108.5577, -439.1210]])


In [ ]:
myF = F_dot[None,:,:]

print(s2.squeeze())
print(s2rot.squeeze())
print(myF[:,:,4:].squeeze())

print(torch.einsum('ik, ijk, ij->i', s2, myF[:,:,4:], s2) - model.k)
print(torch.einsum('ij, ijk, ik->i', s2, myF[:,:,4:], s2) - model.k)
print(torch.einsum('ik, ikj, ij->i', s2, myF[:,:,4:], s2) - model.k)
print(torch.einsum('ij, ikj, ik->i', s2, myF[:,:,4:], s2) - model.k)

print(torch.einsum('ik, ijk, ij->i', s2, myF[:,:,4:], s2rot))
print(torch.einsum('ij, ijk, ik->i', s2, myF[:,:,4:], s2rot))
print(torch.einsum('ik, ikj, ij->i', s2, myF[:,:,4:], s2rot))
print(torch.einsum('ij, ikj, ik->i', s2, myF[:,:,4:], s2rot))

tensor([-0.6361,  0.7716], grad_fn=<SqueezeBackward0>)
tensor([-0.7716, -0.6361], grad_fn=<SqueezeBackward0>)
tensor([[-180.0462, -189.6644],
        [-189.6644, -106.3712]])
tensor([2.2888e-05], grad_fn=<SubBackward0>)
tensor([2.2888e-05], grad_fn=<SubBackward0>)
tensor([2.2888e-05], grad_fn=<SubBackward0>)
tensor([2.2888e-05], grad_fn=<SubBackward0>)
tensor([5.7220e-06], grad_fn=<ViewBackward0>)
tensor([-1.5259e-05], grad_fn=<ViewBackward0>)
tensor([-1.5259e-05], grad_fn=<ViewBackward0>)
tensor([5.7220e-06], grad_fn=<ViewBackward0>)
